<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Загрузка-датасета" data-toc-modified-id="Загрузка-датасета-2.0.0.1"><span class="toc-item-num">2.0.0.1&nbsp;&nbsp;</span>Загрузка датасета</a></span></li><li><span><a href="#Приведение-текстов-к-нижнему-регистру" data-toc-modified-id="Приведение-текстов-к-нижнему-регистру-2.0.0.2"><span class="toc-item-num">2.0.0.2&nbsp;&nbsp;</span>Приведение текстов к нижнему регистру</a></span></li><li><span><a href="#Замена-англицизмов" data-toc-modified-id="Замена-англицизмов-2.0.0.3"><span class="toc-item-num">2.0.0.3&nbsp;&nbsp;</span>Замена англицизмов</a></span></li><li><span><a href="#Замена-числительных" data-toc-modified-id="Замена-числительных-2.0.0.4"><span class="toc-item-num">2.0.0.4&nbsp;&nbsp;</span>Замена числительных</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.0.0.5"><span class="toc-item-num">2.0.0.5&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Подготовка-корпуса" data-toc-modified-id="Подготовка-корпуса-2.0.0.6"><span class="toc-item-num">2.0.0.6&nbsp;&nbsp;</span>Подготовка корпуса</a></span></li></ul></li></ul></li></ul></li></ul></div>

# Импорт библиотек

In [1]:
import pandas as pd
import re
import json
import glob
from sklearn.feature_extraction.text import CountVectorizer

#pickle
import pickle

#spacy
import spacy
import nltk
from nltk.corpus import stopwords 

import numpy as np
from pymystem3 import Mystem
m = Mystem() 
import matplotlib.pyplot as plt
from tqdm import tqdm
import plotly.express as px
import gensim.corpora as corpora
import gensim.models


#Gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.utils import tokenize
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

#vis
import pyLDAvis
import pyLDAvis.gensim_models
import plotly
import plotly.graph_objs as go
import plotly.express as px

#warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Подготовка данных

#### Загрузка датасета

In [2]:
#Прочитаем датасет

df1=pd.read_excel("datasets/russian_news_data.xlsx")

In [3]:
#Выведем пример датасета
df1.sample(20)

,ID,DATE,TIME,TITLE
11657,11658,22.01.2022,21:26:04,Минобороны и МИД Германии оказались недовольны...
17666,17667,07.02.2022,17:13:45,Российские прыгуны с трамплина впервые в истор...
14639,14640,30.01.2022,18:03:40,«Зенит» объявил о приобретении бразильского на...
3153,3154,29.12.2021,03:49:10,Новак назвал долгосрочные контракты условием д...
766,767,21.12.2021,22:50:07,У всех ревакцинированных «Спутником» выявлены ...
14209,14210,29.01.2022,14:28:00,Астения и головные боли названы самыми частыми...
4148,4149,01.01.2022,07:49:36,Александр Овечкин забросил 753-ю шайбу в НХЛ
9885,9886,18.01.2022,04:02:05,"Иммунолог Ярцева призвала вакцинироваться, что..."
10273,10274,19.01.2022,05:56:50,"Назарбаев заявил, что не покидал Казахстан"
14725,14726,30.01.2022,23:43:45,Байден: освобождение похищенного Фрерихса може...


#### Приведение текстов к нижнему регистру

In [4]:
df1['TITLE']=df1['TITLE'].str.lower()

#### Замена англицизмов

In [5]:
%%time
df_engl=pd.read_csv("datasets\Anglicisms(13-02-2022).csv",encoding = 'cp1251',sep=';')
df_engl
df_engl=df_engl.drop(['Unnamed: 0'], axis=1)
df_engl.columns=df_engl.columns.str.lower()
df_engl
df_engl['english']=df_engl['english'].str.lower()
df_engl.sample(10)

Wall time: 3.99 ms


,english,russian
11,qr,куар
78,rai al youm,раиалюм
69,stream,стрим
60,xiaomi,сяоми
61,politico,политико
64,netflix,нетфликс
67,pfizer,пифайзер
0,bloomberg,блумберг
26,comedy club,камеди клаб
29,forbes,форбс


In [6]:
df_engl_dict=df_engl.set_index('english').T.to_dict('records')

<ipython-input-6-5cdb957739c6>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  df_engl_dict=df_engl.set_index('english').T.to_dict('records')


In [7]:
df_engl_dict=df_engl_dict[0]

In [8]:
def replace_from_dict(str,dict):
    for i in (str.split(" ")):
        for j in dict.keys():
            if i==j:
                #print(i)
                str=str.replace(i,dict.get(j))
    return str

In [9]:
#Очистим текст от лишних символов
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Zа-яА-я0-9 ]', ' ', text).split())  
#Выполним проверку
clear_text('hill: сенат сша проголосует по вопросу санкций против «северного потока — 2» до 14 января')

'hill сенат сша проголосует по вопросу санкций против северного потока 2 до 14 января'

In [10]:
df1['TITLE']=df1['TITLE'].apply(lambda x: clear_text(x))
df1['TITLE']=df1['TITLE'].apply(lambda x: replace_from_dict(x,df_engl_dict))

#### Замена числительных

In [11]:
%%time
df_numbers=pd.read_csv("datasets\\Numbers(05-01-2022).csv",encoding = 'cp1251',sep=';')
df_numbers=df_numbers.drop(['Unnamed: 0'], axis=1)
df_numbers=df_numbers.dropna()
df_numbers.sample(10)

Wall time: 9.98 ms


,number,word
176,1050,тысяча пятьдесят
66,67,шестьдесят семь
206,1990,тысяча девятьсот девяносто
237,2028,две тысячи двадцать восемь
194,1945,тысяча девятьсот сорок пять
99,100,сто
97,98,девяносто восемь
31,32,тридцать два
149,150,сто пятьдесят
174,950,девятьсот пятьдесят


In [12]:
df_numbers=df_numbers.set_index('number').T.to_dict('records')
df_numbers_dict=df_numbers[0]

<ipython-input-12-4f619f413480>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  df_numbers=df_numbers.set_index('number').T.to_dict('records')


In [13]:
df1['TITLE']=df1['TITLE'].apply(lambda x: replace_from_dict(x,df_numbers_dict))

#### Лемматизация

In [14]:
#Напишем фукнцию лемматизации
def lemmatization(texts):
    nlp = spacy.load("ru_core_news_sm")
    texts_out = []
    for text in tqdm(texts):
        doc = nlp(text)
        new_text = []
        for token in doc:
            new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

In [15]:
%%time
lemmatized_texts=lemmatization(df1['TITLE'])
print(lemmatized_texts[0:10])

100%|███████████████████████████████████████████████████████████████████████████| 19576/19576 [02:33<00:00, 127.81it/s]

['из шахта в кузбассе после авария эвакуировать сто двадцать горняк', 'роспотребнадзор изучать возможность введение штраф за отказ от вакцинация', 'грушко предложение россия по гарантия безопасность не содержать скрытый повестка', 'министр оборона фрг призвать направить санкция против руководство россия', 'карнаухов и маннинен стать хороший игрок матч россия финляндия', 'минобороны показать видео уничтожение боевик в сирия дрон ланцет в два тысяча двадцать год', 'умереть режиссёр фильм мы из будущее андрей малюков', 'мвд объяснить сбой в регистрационный система гибдд затопление серверный помещение', 'кристина резцова завоевать бронза в масс старт на этап кубок мир по биатлон в анси', 'американский сенатор поругаться из за северный поток два']
Wall time: 2min 34s


#### Подготовка корпуса

In [16]:
corpus=np.array(lemmatized_texts)
len(corpus)

19576

Сериалазируем итоговый корпус:

In [17]:
with open("corpus_preprocessed.pkl", "wb") as f:
    pickle.dump(corpus, f)